In [11]:
import pandas as pd
import json
from tqdm import tqdm

In [5]:
df = pd.read_csv("data/mentions-month.gzip", compression='gzip')
df

GLOBALEVENTID   EventTimeDate  MentionTimeDate  MentionType  \
0           803346281  20181117110000   20181117110000            1   
1           803296669  20181117043000   20181117110000            1   
2           803346663  20181117110000   20181117110000            1   
3           803281180  20181117024500   20181117110000            1   
4           796045333  20181020014500   20181020180000            1   
5           796154309  20181020160000   20181020201500            1   
6           796154322  20181020160000   20181020201500            1   
7           796262881  20181021074500   20181021074500            1   
8           796263317  20181021074500   20181021074500            1   
9           797945242  20181027110000   20181027110000            1   
10          797893728  20181027044500   20181027113000            1   
11          796110168  20181020103000   20181020133000            1   
12          796020716  20181020000000   20181020014500            1   
13          796020716  20181020000000   20181020014500            1   
14          796045333  20181020014500   20181020014500            1   
15          796059494  20181020031500   20181020031500            1   
16          796154309  20181020160000   20181020160000            1   
17          796154322  20181020160000   20181020160000            1   
18          797815824  20181026203000   20181026203000            1   
19          797898531  20181027050000   20181027050000            1   
20          797921048  20181027080000   20181027080000            1   
21          797921084  20181027080000   20181027080000            1   
22          797921085  20181027080000   20181027080000            1   
23          797921280  20181027080000   20181027080000            1   
24          796101943  20181020090000   20181020090000            1   
25          796101943  20181020090000   20181020140000            1   
26          796213953  20181021003000   20181021181500            1   
27          796297564  20181021123000   20181021123000            1   
28          796419000  20181022050000   20181022050000            1   
29          796527418  20181022143000   20181022143000            1   
...               ...             ...              ...          ...   
749697      798297232  20181029100000   20181029100000            1   
749698      798297933  20181029100000   20181029100000            1   
749699      798300109  20181029101500   20181029101500            1   
749700      798300793  20181029101500   20181029101500            1   
749701      798300793  20181029101500   20181029110000            1   
749702      798300793  20181029101500   20181029124500            1   
749703      798300793  20181029101500   20181029124500            1   
749704      798334408  20181029130000   20181029130000            1   
749705      798335196  20181029130000   20181029130000            1   
749706      802370137  20181114000000   20181114093000            1   
749707      797260808  20181025000000   20181025013000            1   
749708      796655935  20181023001500   20181023171500            1   
749709      796655935  20181023001500   20181023171500            1   
749710      796887100  20181023171500   20181023171500            1   
749711      796882662  20181023171500   20181023171500            1   
749712      796884513  20181023171500   20181023171500            1   
749713      796881552  20181023171500   20181023171500            1   
749714      796880636  20181023171500   20181023171500            1   
749715      796879463  20181023171500   20181023171500            1   
749716      796884179  20181023171500   20181023171500            1   
749717      796884189  20181023171500   20181023171500            1   
749718      796879101  20181023171500   20181023171500            1   
749719      796881995  20181023171500   20181023171500            1   
749720      796884509  20181023171500   20181023171500            1   
749721      796884716  201810231

In [6]:
# sanity check : we only have articles (MentionType 1)
df[df.MentionType!=1]

Empty DataFrame
Columns: [GLOBALEVENTID, EventTimeDate, MentionTimeDate, MentionType, MentionSourceName, MentionIdentifier, SentenceID, Actor1CharOffset, Actor2CharOffset, ActionCharOffset, InRawText, Confidence, MentionDocLen, MentionDocTone, MentionDocTranslationInfo, Extras]
Index: []

In [7]:
# distribution of nb of articles by sources
nb_articles = df.groupby("MentionSourceName").size()
nb_articles.describe()

count       50.000000
mean     14994.540000
std      10704.597264
min       6754.000000
25%       7575.250000
50%      10240.000000
75%      19197.000000
max      57083.000000
dtype: float64

In [8]:
sources = set(df.MentionSourceName)

In [9]:
domains = { ".".join(s.split(".")[1:]) for s in sources }
len(domains)

19

In [13]:
threshold = 10
nodes = []
edges = []
new_sources = sources

for source1 in tqdm(sources):
    nodes.append({ 'id' : source1, 'size' : int(nb_articles[source1]), 'group' : ".".join(source1.split(".")[1:]) }) 
    events1 = set(df[df.MentionSourceName==source1].GLOBALEVENTID)
    new_sources = new_sources - {source1}
    
    for source2 in tqdm(new_sources):
        if source1==source2:
            continue
            
        events2 = set(df[df.MentionSourceName==source2].GLOBALEVENTID)
        w = len(events1 & events2)
        
        if w > threshold:
            edges.append({'source': source1, 'target': source2, 'weight': w})  

 51%|█████     | 24/47 [00:00<00:00, 28.24it/s]



KeyboardInterrupt: 

In [30]:
nodes

[{'id': 'haberler.com', 'size': 747, 'group': 'com'},
 {'id': 'dw.com', 'size': 564, 'group': 'com'},
 {'id': 'washingtonpost.com', 'size': 497, 'group': 'com'},
 {'id': 'yahoo.com', 'size': 1759, 'group': 'com'},
 {'id': 'wkrb13.com', 'size': 458, 'group': 'com'},
 {'id': 'regnum.ru', 'size': 683, 'group': 'ru'},
 {'id': 'ziarelive.ro', 'size': 1480, 'group': 'ro'},
 {'id': 'reuters.com', 'size': 2404, 'group': 'com'},
 {'id': 'nzherald.co.nz', 'size': 443, 'group': 'co.nz'},
 {'id': 'ria.ru', 'size': 565, 'group': 'ru'},
 {'id': 'tribunnews.com', 'size': 1501, 'group': 'com'},
 {'id': 'wickedlocal.com', 'size': 617, 'group': 'com'},
 {'id': 'reliefweb.int', 'size': 378, 'group': 'int'},
 {'id': 'zazoom.it', 'size': 1274, 'group': 'it'},
 {'id': 'xinhuanet.com', 'size': 591, 'group': 'com'},
 {'id': 'zocalo.com.mx', 'size': 371, 'group': 'com.mx'},
 {'id': 'sfgate.com', 'size': 537, 'group': 'com'},
 {'id': 'uol.com.br', 'size': 597, 'group': 'com.br'},
 {'id': 'bbc.com', 'size': 418,

In [12]:
len(edges)

976

In [32]:
network = {'nodes': nodes, 'edges': edges}

In [33]:
with open("data/network.json", "w") as f:
    json.dump(network, f, indent=2)